In [45]:
pip install dotenv pypdf langchain langchain-community langchain-openai langchain-pinecone docx2txt tiktoken  -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [46]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

In [47]:
pip install wikipedia -q 


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [48]:
def load_document(file):
    import os

    name, extension = os.path.splitext(file)

    if extension == ".pdf":
        from langchain_classic.document_loaders import PyPDFLoader

        print(f"Loading {file}")
        loader = PyPDFLoader(file)
    elif extension == ".docx":
        from langchain_classic.document_loaders import Docx2txtLoader

        loader = Docx2txtLoader(file)
    else:
        print("Document format is not supported")
        return None

    data = loader.load()
    return data


# wikipedia


def load_from_wikipedia(query, lang="en", load_max_docs=2):
    from langchain_classic.document_loaders import WikipediaLoader

    loader = WikipediaLoader(query=query, lang=lang, load_max_docs=load_max_docs)
    data = loader.load()
    return data

In [49]:
def chunk_data(data, chunk_size=256):
    from langchain_classic.text_splitter import RecursiveCharacterTextSplitter

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=0
    )
    chunks = text_splitter.split_documents(data)
    return chunks

In [50]:
def print_embedding_cost(texts):
    import tiktoken

    enc = tiktoken.encoding_for_model("text-embedding-ada-002")
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f"Total Tokens: {total_tokens}")
    print(f"Embedding Cost in USD: {total_tokens / 1000 * 0.00002:.6f}")

### Embedding and uploading to a Vector Database (Pinecone)

In [51]:
def insert_or_fetch_embeddings(index_name, chunks):
    import pinecone

    # from langchain_community.vectorstores import Pinecone
    from langchain_pinecone.vectorstores import Pinecone
    from langchain_openai import OpenAIEmbeddings
    from pinecone import ServerlessSpec

    pc = pinecone.Pinecone()
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1536)

    if index_name in pc.list_indexes().names():
        print(f"Index {index_name} already exists. Loading embeddings...", end="")
        vector_store = Pinecone.from_existing_index(index_name, embeddings)
        print("Ok")
    else:
        print(f"Creating index {index_name} and embeddings...", end="")
        pc.create_index(
            name=index_name,
            dimension=1536,
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1"),
        )
        vector_store = Pinecone.from_documents(
            chunks, embeddings, index_name=index_name
        )
        print("Ok")
        return vector_store

In [52]:
def delete_pinecone_index(index_name="all"):
    import pinecone

    pc = pinecone.Pinecone()
    if index_name == "all":
        indexes = pc.list_indexes().names()
        print("Deleting all indexes...")
        for index in indexes:
            pc.delete_index(index)
        print("Ok")
    else:
        print(f"Deleting index {index_name}...", end="")
        pc.delete_index(index_name)
        print("Ok")

In [59]:
def ask_and_get_answer(vector_store, q):
    from langchain_openai import ChatOpenAI
    from langchain_classic.chains.retrieval_qa.base import RetrievalQA

    llm = ChatOpenAI(temperature=1)

    retriever = vector_store.as_retriever(
        search_type="similarity", search_kwargs={"k": 5}
    )

    chain = RetrievalQA.from_chain_type(
        llm=llm, chain_type="stuff", retriever=retriever
    )
    answer = chain.invoke(q)
    return answer


## Running code

In [54]:
data = load_document("pdfs/guia_lgpd.pdf")
# print(data[1].page_content)

print(f"You have {len(data)} pages in your data")

Loading pdfs/guia_lgpd.pdf
You have 69 pages in your data


In [8]:
# data = load_from_wikipedia("Gemini-AI")
# print(data[0].page_content)

In [57]:
chunks = chunk_data(data)
print(len(chunks))
print(chunks[10].page_content)

929
HISTÓRICO DE VERSÕES
DATA VERSÃO DESCRIÇÃO AUTOR
23/03/2020 1.0 Primeira versão do Guia de Boas Práticas. Equipe Técnica de 
Elaboração
14/08/2020 2.0
Segunda versão do Guia de Boas Práticas 
considerando as sugestões enviadas pelo:


In [42]:
print_embedding_cost(chunks)

Total Tokens: 52383
Embedding Cost in USD: 0.001048


In [27]:
delete_pinecone_index()

Deleting all indexes...
Ok


In [ ]:
index_name = "askadocument"
vector_store = insert_or_fetch_embeddings(index_name)

Creating index askadocument and embeddings...Ok


In [60]:
q = "Sobre o que o documento fala?"
answer = ask_and_get_answer(vector_store, q)
print(answer)

{'query': 'Sobre o que o documento fala?', 'result': 'O documento parece estar discutindo as obrigações relacionadas ao tratamento de dados pessoais, transparência ativa, meios de acesso à informação, gestão de documentos e segurança da informação, de acordo com a LGPD (Lei Geral de Proteção de Dados). Além disso, menciona a aplicação de normas e padrões, como a ABNT NBR ISO/IEC 15489:2018, e a relação com o Poder Executivo Federal e o CCD (Conselho de Controle de Atividades Financeiras) em relação às atividades-fim e à gestão de documentos de arquivo.'}


In [68]:
import time

i = 1
print("Write Quit or Exit to quit.")
while True:
    q = input(f"Question #{i}: ")
    i = i + 1
    if q.lower() in ["quit", "exit"]:
        print("Quitting... bye")
        time.sleep(2)
        break

    answer = ask_and_get_answer(vector_store, q)
    print(f"\nAnswer: {answer['result']}")
    print(f"\n {'-' * 50} \n")

Write Quit or Exit to quit.

Answer: A ePING é um conjunto de padrões de interoperabilidade de Governo Eletrônico que define premissas, políticas e especificações técnicas para a utilização de Tecnologia de Informação e Comunicação (TIC) visando a interconexão, segurança, meios de acesso, organização e intercâmbio de informações, e áreas de integração para o Governo Eletrônico. Esses padrões visam garantir a interoperabilidade dos serviços de Governo Eletrônico com os demais Poderes, esferas de governo e a sociedade em geral. Mais informações podem ser encontradas no site http:/ /eping.governoeletronico.gov.br/

 -------------------------------------------------- 

Quitting... bye
